# Demo 1: Quantum Interference #

## Introduction ##

One of the most striking features demonstrated by quantum matter is that of *interference*, a consequence of the wavelike and coherent nature of quantum systems.  Interference is the fundamental phenomena that enables the burgeoning field of quantum sensing.  Many of the world's most sensitive measurements of gravitational waves, accelerations, rotations, gravity, magnetic fields and time all leverage quantum interference and the interferometry techniques built upon it.  We can start exploring this vast and fascinating topic using Oqtant QMS. 

To demonstrate interference, we need to split the quantum matter into $\geq 1$ distinct spatial regions, release the trapping potential so that the spatially-separated samples can expand and overlap, and detect the interference by taking an image.  With Barrier objects and time-of-flight imaging, we have all the tools necessary to accomplish this demonstration.

Note: For further reading, see e.g. the 1997 paper [Observation of Interference Between Two Bose Condensates](https://www.rle.mit.edu/cua_pub/ketterle_group/Projects_1997/Pubs_97/andr97-Science_int.pdf) which used an experimental setup very similar to the hardware accessible by Oqtant QMS. 

### Imports and user authentication ###

In [ ]:
from oqtant.schemas.quantum_matter import QuantumMatterFactory

qmf = QuantumMatterFactory()
qmf.get_login()

In [ ]:
qmf.get_client()

## Split quantum matter into two components ##

First, we will create a barrier that will be used to split our atom ensemble into two distinct spatial regions. The goal is to gently separate the cloud by slowly increasing the barrier height so that the quantum matter isn't heated or excited in the process. On the other hand, the process can't be too slow because the quantum matter clouds on either side of the barrier need to remain coherent. After splitting the cloud, the barrier and overall (magnetic) trapping potential is turned off to allow the clouds to recombine as they fall under gravity.

In [ ]:
barrier = qmf.create_barrier(
    positions=[0, 0],  # fine tune to split the cloud into roughly equal parts
    heights=[0, 12],
    widths=[4, 4],  # adjust as free parameter to optimize observed interference
    times=[0, 6],
)

# barrier.evolve(duration=2) # can also hold the barrier for some time (after ramp-on) to explore coherence time

barrier.show_dynamics()

To explore the emergence and evolution of quantum interference, the barrier height, width, shape, ramp-on, and optional hold-time (after ramping on) can all be varied.  Another useful parameter to explore is the temperature of the quantum matter before the barrier is introduced.

## In-trap image to verify cloud has been split ##

Next, we create a QuantumMatter object to image the atom ensemble in-trap.

In [ ]:
matter_in_trap = qmf.create_quantum_matter(
    temperature=25,  # another free parameter available for optimizing observed interference
    lifetime=barrier.death,
    barriers=[barrier],
    image="IN_TRAP",
)

matter_in_trap.show_potential(times=[0, 3, 6], ylimits=[-2, 40])

We next submit our QuantumMatter object to the client to create a job, wait for the job to execute, and fetch the results:

In [ ]:
matter_in_trap.submit(track=True)

### Plot the resulting in-trap image ###

In [ ]:
matter_in_trap.get_result()
matter_in_trap.output.plot_it(figsize=(6, 6))

## Observe interference in time-of-flight imaging ##

Next, let's look at the atom ensemble after releasing it from the trap. As the two regions of the quantum matter expand during the (adjustable) time of flight, they begin to spatially overlap, which results in interference.

In [ ]:
tofs = [5, 7, 9, 11, 13, 15, 17, 19]
tof_matters = []
for tof in tofs:
    tof_matters.append(
        qmf.create_quantum_matter(
            temperature=25,
            barriers=[barrier],
            lifetime=barrier.death,
            image="TIME_OF_FLIGHT",
            time_of_flight=tof,
            name="interference w/ tof = " + str(tof) + " ms",
        )
    )

Submit to QMS and retrieve results, placing the resulting jobs (with outputs) in a list

In [ ]:
for matter in tof_matters:
    matter.submit()

Once our sequence of interference jobs are complete and fetched, we can observe the overlapping quantum matter.  If your inputs above are tuned correctly, you should observe the onset of quantum interference! 

In [ ]:
import matplotlib.pyplot as plt

for matter in tof_matters:
    matter.get_result()
    matter.output.plot_tof(figsize=(6, 6))

The cross sections of the overlapping quantum matter clouds is often particularly revealing:

In [ ]:
for matter in tof_matters:
    matter.output.plot_slice(axis="x")